# 9. 파서 조합기 라이브러리

함수적 설계 과정에 대한 영감을 제공하는데 초점을 둔다.

### 파서 조합기 VS 파서 생성기

## 9.1 대수적 설계

#### 하나의 문자 'a'를 인식하는 파서

In [ ]:
def char(c: Char): Parser[Char]

#### 파서 실행을 위한 함수

In [ ]:
def run[A](p: Parser[A])(input: String): Either[ParseError,A] 

#### 인터페이스

In [ ]:
trait Parsers[ParseError, Parser[+_]] {
    def run[A](p: Parser[A])(input: String): Either[ParseError,A]
    def char(c: Char): Parser[Char]
}

In [ ]:
run(char(c))(c.toString) == Right(c)

#### "abracadabra"와 같은 문자열을 위한 파서

In [ ]:
def string(s: String): Parser[String]

#### "abra" 또는 "cadabra"를 인식

In [ ]:
def orString(s1: string, s2: String): Parser[String]

In [ ]:
def or[A](s1: Parser[A], s2: Parser[A]): Parser[A]

#### or 조합기를 위한 중위 구문(infix syntax) 추가.

In [ ]:
trait Parsers[ParserError, Parser[+_]] { self =>
    def or[A](s1: Parser[A], s2: Parser[A]): Parser[A]
    implicit def string(s: String): Parser[String]
    implicit def operators[A](p: Parser[A]) = ParserOps[A](p)
    implicit def asStringParser[A](a: A)(implicit f: A => Parser[String]):
        ParserOps[String] = ParserOps(f(a))
    
    case class ParserOps[A](p: Parser[A]) {
        def |[B>:A](p2: Parser[B]): Parser[B] = self.or(p,p2)
        def or[B>:A](p2: Parser[B]): Parser[B] = self.or(p,p2)
    }
}

#### 파서가 되풀이 되는 경우를 위한 조합기

In [ ]:
def listOfN[A](n: Int, p: Parser[A]): Parser[List[A]]

## 9.2 가능한 대수 하나

#### 0개 이상의 문자 'a'를 인식해서 그 개수를 돌려주는 파서

In [ ]:
def many[A](p: Parser[A]): Parser[List[A]]

In [ ]:
def map[A,B](a: Parser[A])(f: A => B): Parser[B]

In [ ]:
map(many(char('a')))(_.size)

In [ ]:
val numA: Parser[Int] = char('a').many.map(_.size)

### 9.2.1 슬라이싱

In [ ]:
def slice[A](p: Parser[A]): Parser[String]

#### A 파서가 성공하면 B 파서를 실행

In [ ]:
def product[A,B](p: Parser[A], p2: Parser[B]): Parser[(A,B)]

## 9.3 문맥 민감성의 처리

In [ ]:
def flatMap[A,B](p: Parser[A])(f: A => Parser[B]): Parser[B]

## 9.4 JSON 파서 작성

In [ ]:
def jsonParser[Err,Parser[+_]](P: Parsers[Err,Parser]): Parser[JSON] = {
    import P._
    val spaces = char(' ').many.slice
    ...
}

### 9.4.1 JSON 서식

In [ ]:
trait JSON
object JSON {
    case object JNull extends JSON
    case class JNumber(get: Double) extends JSON
    case class JString(get: String) extends JSON
    case class JBool(get: Boolean) extends JSON
    case class JArray(get: IndexedSeq[JSON]) extends JSON
    case class JObject(get: Map[String, JSON]) extends JSON
}

### 9.4.2 JSON 파서

## 9.5 오류 보고

### 9.5.1 가능한 설계 하나

In [ ]:
def label[A](msg: String)(p: Parser[A]): Parser[A]

In [ ]:
case class Location(input: String, offset: Int = 0) {
    lazy val line = input.slice(0, offset+1).count(_ == '\n') + 1
    lazy val col = input.slice(0, offset+1).lastIndexOf('\n') match {
        case -1 => offset + 1
        case lineStart => offset - lineStart
    }
}


def errorLocation(e: ParseError): Location
def errorMessage(e: ParseError): String

In [ ]:
- 전체 입력, 오프셋, 행 번호, 열 번호
- 행 번호와 열 번호는 게으르게 계산됨.